In [ ]:
! pip install pytorch-lightning~=1.5.0
! pip install datasets~=2.1.0
! pip install transformers
! pip install torchmetrics~=0.7.0
! pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from transformers import AutoTokenizer, AutoModel, pipeline
from datasets import load_dataset
from datasets import load_metric
import torch
import torch.nn as nn
from transformers import BertTokenizer, BertModel

from transformers import RobertaTokenizerFast,RobertaForSequenceClassification, AdamW, TrainingArguments, Trainer,AutoModelForSequenceClassification
dataset = load_dataset("McGill-NLP/FaithDial")
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',max_length=256)
itemsUnique=[]
for items in dataset["train"]["BEGIN"]:
    for val in items:
        if val not in itemsUnique:
            itemsUnique.append(val)

id2label = {idx:itemsUnique for idx, itemsUnique in enumerate(itemsUnique)}
label2id = {itemsUnique:idx for idx, itemsUnique in enumerate(itemsUnique)}
print(id2label)

binary_dataset = dataset.map(lambda examples: {"original_response": examples["response"]  if examples["original_response"] == None else examples["original_response"]})
import numpy as np
def preprocess_function(examples):
    encoding=tokenizer(examples["knowledge"], examples["original_response"], truncation=True,padding='max_length')
    tempLables=examples["BEGIN"]
    lables=np.zeros(len(label2id))
    for tempLable in tempLables:
        index=label2id[tempLable]
        lables[index]=1
    encoding["labels"]=lables
    
    return encoding
encoded_dataset = binary_dataset.map(preprocess_function,remove_columns=binary_dataset['train'].column_names)
print(dataset)

  0%|          | 0/7 [00:00<?, ?it/s]

{0: 'Hallucination', 1: 'Entailment', 2: 'Uncooperative', 3: 'Generic'}


  0%|          | 0/3539 [00:00<?, ?ex/s]

  0%|          | 0/1716 [00:00<?, ?ex/s]

  0%|          | 0/1823 [00:00<?, ?ex/s]

...

  0%|          | 0/18357 [00:00<?, ?ex/s]

  0%|          | 0/3417 [00:00<?, ?ex/s]

  0%|          | 0/1666 [00:00<?, ?ex/s]

  0%|          | 0/1751 [00:00<?, ?ex/s]

DatasetDict({
    test: Dataset({
        features: ['dialog_idx', 'response', 'original_response', 'history', 'knowledge', 'BEGIN', 'VRM'],
        num_rows: 3539
    })
    test_random_split: Dataset({
        features: ['dialog_idx', 'response', 'original_response', 'history', 'knowledge', 'BEGIN', 'VRM'],
        num_rows: 1716
    })
    test_topic_split: Dataset({
        features: ['dialog_idx', 'response', 'original_response', 'history', 'knowledge', 'BEGIN', 'VRM'],
        num_rows: 1823
    })
    train: Dataset({
        features: ['dialog_idx', 'response', 'original_response', 'history', 'knowledge', 'BEGIN', 'VRM'],
        num_rows: 18357
    })
    validation: Dataset({
        features: ['dialog_idx', 'response', 'original_response', 'history', 'knowledge', 'BEGIN', 'VRM'],
        num_rows: 3417
    })
    valid_random_split: Dataset({
        features: ['dialog_idx', 'response', 'original_response', 'history', 'knowledge', 'BEGIN', 'VRM'],
        num_rows: 1666
    

In [ ]:
print(dataset["train"][1])
print(encoded_dataset["train"])

{'dialog_idx': 0, 'response': 'Well, I know that more people started using it after some restrictions on internet use were lifited in 1995', 'original_response': 'It used to be restricted, but around 1995, the restricted were lifted and commercial use of it began', 'history': ['Can you imagine the world without internet access?', 'Yeah, but once the access to the internet was a rare thing. do you remember?', 'I do. What else can you tell me ?'], 'knowledge': 'Use by a wider audience only came in 1995 when restrictions on the use of the Internet to carry commercial traffic were lifted.', 'BEGIN': ['Hallucination', 'Entailment'], 'VRM': ['Edification']}
Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 18357
})


In [ ]:
# hyperparameters
MAX_LEN = 256
TRAIN_BATCH_SIZE = 10
VALID_BATCH_SIZE = 10
EPOCHS = 2
LEARNING_RATE = 1e-03

In [ ]:
from transformers import default_data_collator
train_data_loader = torch.utils.data.DataLoader(encoded_dataset["train"], 
    batch_size=TRAIN_BATCH_SIZE,
    shuffle=True,
    num_workers=0,
    collate_fn=default_data_collator
)

In [ ]:
val_data_loader = torch.utils.data.DataLoader(encoded_dataset["validation"], 
    batch_size=VALID_BATCH_SIZE,
    shuffle=False,
    num_workers=0,
    collate_fn=default_data_collator
)
     

In [ ]:
# from transformers import default_data_collator
# data_collator = DataCollatorWithPadding(tokenizer)
# train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=16, 
# collate_fn=data_collator)

In [ ]:
# train_featuresain_labels = next(iter(train_data_loader))
# print(train_featuresain_labels["input_ids"])
# print("attention_mask")
# print(train_featuresain_labels["attention_mask"])
# print("labels")
# print(train_featuresain_labels["labels"])

In [ ]:
# for batch_idx, data in enumerate(train_data_loader):
#   #print('yyy epoch', batch_idx)
#   ids = data['input_ids'].to(device, dtype = torch.long)
#   mask = data['attention_mask'].to(device, dtype = torch.long)
#   token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
#   targets = data['labels'].to(device, dtype = torch.float)
#   break

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')


In [ ]:
class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.bert_model = BertModel.from_pretrained('bert-base-uncased', return_dict=True)
        self.dropout = torch.nn.Dropout(0.3)
        self.linear = torch.nn.Linear(768, 4)
    
    def forward(self, input_ids, attn_mask, token_type_ids):
        output = self.bert_model(
            input_ids, 
            attention_mask=attn_mask, 
            token_type_ids=token_type_ids
        )
        output_dropout = self.dropout(output.pooler_output)
        output = self.linear(output_dropout)
        return output

model = BERTClass()
model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BERTClass(
  (bert_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [ ]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [ ]:
val_targets=[]
val_outputs=[]

In [ ]:
def train_model(n_epochs, training_loader, validation_loader, model, 
                optimizer, checkpoint_path, best_model_path):
   
  # initialize tracker for minimum validation loss
  valid_loss_min = np.Inf
   
 
  for epoch in range(1, n_epochs+1):
    train_loss = 0
    valid_loss = 0

    model.train()
    print('############# Epoch {}: Training Start   #############'.format(epoch))
    for batch_idx, data in enumerate(training_loader):
        #print('yyy epoch', batch_idx)
        ids = data['input_ids'].to(device, dtype = torch.long)
        mask = data['attention_mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['labels'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids)

        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)
        #if batch_idx%5000==0:
         #   print(f'Epoch: {epoch}, Training Loss:  {loss.item()}')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print('before loss data in training', loss.item(), train_loss)
        train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.item() - train_loss))
        print('after loss data in training', loss.item(), train_loss)
    
    print('############# Epoch {}: Training End     #############'.format(epoch))
    
    print('############# Epoch {}: Validation Start   #############'.format(epoch))
    ######################    
    # validate the model #
    ######################
 
    model.eval()
   
    with torch.no_grad():
      for batch_idx, data in enumerate(validation_loader, 0):
            ids = data['input_ids'].to(device, dtype = torch.long)
            mask = data['attention_mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['labels'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)

            loss = loss_fn(outputs, targets)
            valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.item() - valid_loss))
            val_targets.extend(targets.cpu().detach().numpy().tolist())
            val_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())

      print('############# Epoch {}: Validation End     #############'.format(epoch))
      # calculate average losses
      #print('before cal avg train loss', train_loss)
      train_loss = train_loss/len(training_loader)
      valid_loss = valid_loss/len(validation_loader)
      # print training/validation statistics 
      print('Epoch: {} \tAvgerage Training Loss: {:.6f} \tAverage Validation Loss: {:.6f}'.format(
            epoch, 
            train_loss,
            valid_loss
            ))
      
      # create checkpoint variable and add important data
      checkpoint = {
            'epoch': epoch + 1,
            'valid_loss_min': valid_loss,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict()
      }
        
        # save checkpoint
      # save_ckp(checkpoint, False, checkpoint_path, best_model_path)
        
      ## TODO: save the model if validation loss has decreased
      if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,valid_loss))
        # save checkpoint as best model
        # save_ckp(checkpoint, True, checkpoint_path, best_model_path)
        valid_loss_min = valid_loss

    print('############# Epoch {}  Done   #############\n'.format(epoch))

  return model

In [ ]:
ckpt_path = "/content/drive/MyDrive/datasets/multi-label/curr_ckpt"
best_model_path = "/content/drive/MyDrive/datasets/multi-label/best_model.pt"

In [ ]:
trained_model = train_model(EPOCHS, train_data_loader, val_data_loader, model, optimizer, ckpt_path, best_model_path)


Streaming output truncated to the last 5000 lines.
after loss data in training 0.16303090751171112 0.26010930616471756
before loss data in training 0.11642356216907501 0.26010930616471756
after loss data in training 0.11642356216907501 0.25998764169054683
before loss data in training 0.24377799034118652 0.25998764169054683
after loss data in training 0.24377799034118652 0.2599739279415203
before loss data in training 0.2032672017812729 0.2599739279415203
after loss data in training 0.2032672017812729 0.25992599326175675
before loss data in training 0.1479438692331314 0.25992599326175675
after loss data in training 0.1479438692331314 0.25983141376511093
before loss data in training 0.26749229431152344 0.25983141376511093
after loss data in training 0.26749229431152344 0.2598378786432092
before loss data in training 0.14502818882465363 0.2598378786432092
after loss data in training 0.14502818882465363 0.2597410745202593
before loss data in training 0.2685457170009613 0.2597410745202593
a

In [ ]:
print(type(trained_model))

<class '__main__.BERTClass'>


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
path = F"/content/gdrive/My Drive/begin_classification_bert.pt"

In [ ]:
# torch.save(trained_model, path)

In [ ]:
print(encoded_dataset["test"])

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 3539
})


In [ ]:
model.eval()

BERTClass(
  (bert_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [ ]:
# print(type(encoded_dataset["test"]["input_ids"][0]))
# temp=torch.IntTensor(encoded_dataset["test"]["input_ids"][0])
# print(temp)

def getOutput(i_val):
  inupt_ids = torch.IntTensor(encoded_dataset["test"]["input_ids"][i_val])
  inupt_ids = inupt_ids.unsqueeze(0).to(device) # Add a batch dimension
  attention_mask = torch.IntTensor(encoded_dataset["test"]["attention_mask"][i_val])
  attention_mask = attention_mask.unsqueeze(0).to(device) # Add a batch dimension
  token_type_ids = torch.IntTensor(encoded_dataset["test"]["token_type_ids"][i_val])
  token_type_ids = token_type_ids.unsqueeze(0).to(device) # Add a batch dimension
  output=trained_model(inupt_ids,attention_mask,token_type_ids)
  final_output = torch.sigmoid(output).cpu().detach().numpy().tolist()
  return final_output

In [ ]:
final_outputs=[]
for i in range(0,len(encoded_dataset["test"])):
  final_output=getOutput(i)
  final_outputs.append(final_output[0])
  print(final_output)

[[0.9940404295921326, 0.7814093828201294, 0.0027154255658388138, 0.002204085001721978]]
[[0.023955848067998886, 0.9817574620246887, 0.462361603975296, 0.016841810196638107]]
[[0.06825467199087143, 0.9851750731468201, 0.21617521345615387, 0.007746089715510607]]
[[0.6271509528160095, 0.9730783700942993, 0.042818550020456314, 0.0037544448859989643]]
[[0.9912410974502563, 0.9544562101364136, 0.003567634616047144, 0.001997680403292179]]
[[0.9897984266281128, 0.9578798413276672, 0.0035554252099245787, 0.0015703876269981265]]
[[0.9899064898490906, 0.9525572657585144, 0.0034888992086052895, 0.0017090791370719671]]
[[0.9908685684204102, 0.9440443515777588, 0.0031952171120792627, 0.0016578377690166235]]
[[0.9895157814025879, 0.9613336324691772, 0.00360406213440001, 0.00175891921389848]]
[[0.973995566368103, 0.9770734906196594, 0.00426468113437295, 0.0015361044788733125]]
[[0.1825801283121109, 0.9875714778900146, 0.08886487036943436, 0.0034462790936231613]]
[[0.22467713057994843, 0.97267466783523

In [ ]:
print(final_outputs)

[[0.9940404295921326, 0.7814093828201294, 0.0027154255658388138, 0.002204085001721978], [0.023955848067998886, 0.9817574620246887, 0.462361603975296, 0.016841810196638107], [0.06825467199087143, 0.9851750731468201, 0.21617521345615387, 0.007746089715510607], [0.6271509528160095, 0.9730783700942993, 0.042818550020456314, 0.0037544448859989643], [0.9912410974502563, 0.9544562101364136, 0.003567634616047144, 0.001997680403292179], [0.9897984266281128, 0.9578798413276672, 0.0035554252099245787, 0.0015703876269981265], [0.9899064898490906, 0.9525572657585144, 0.0034888992086052895, 0.0017090791370719671], [0.9908685684204102, 0.9440443515777588, 0.0031952171120792627, 0.0016578377690166235], [0.9895157814025879, 0.9613336324691772, 0.00360406213440001, 0.00175891921389848], [0.973995566368103, 0.9770734906196594, 0.00426468113437295, 0.0015361044788733125], [0.1825801283121109, 0.9875714778900146, 0.08886487036943436, 0.0034462790936231613], [0.22467713057994843, 0.9726746678352356, 0.13460

In [ ]:
final_output = torch.sigmoid(output).cpu().detach().numpy().tolist()
print(final_output)

[[0.06825467199087143, 0.9851750731468201, 0.21617521345615387, 0.007746089715510607]]


In [ ]:
print(len(final_outputs))
prediction_converted=[]
for prediction in final_outputs:
  indiarray=[]
  for indi in prediction:
    if indi>0.5:
      indiarray.append(1.0)
    else:
      indiarray.append(0.0)
  prediction_converted.append(indiarray)
print(prediction_converted)


3539
[[1.0, 1.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [1.0

In [ ]:
count=0
total=0
for i in range(0,len(prediction_converted)):
  if prediction_converted[i]==encoded_dataset['test']["labels"][i]:
    count+=1
  total+=1
print("correct count",count)
print("total count",total)

correct count 2308
total count 3539


In [ ]:
print("accuracy",count/total)

accuracy 0.652161627578412


In [ ]:
print(encoded_dataset["test"]["labels"][2])

[0.0, 1.0, 0.0, 0.0]


In [ ]:
  i_val=1
  inupt_ids = torch.IntTensor(encoded_dataset["test"]["input_ids"][i_val])
  inupt_ids = inupt_ids.unsqueeze(0).to(device) # Add a batch dimension
  attention_mask = torch.IntTensor(encoded_dataset["test"]["attention_mask"][i_val])
  attention_mask = attention_mask.unsqueeze(0).to(device) # Add a batch dimension
  token_type_ids = torch.IntTensor(encoded_dataset["test"]["token_type_ids"][i_val])
  token_type_ids = token_type_ids.unsqueeze(0).to(device) # Add a batch dimension
  output=model(inupt_ids,attention_mask,token_type_ids)
  final_output = torch.sigmoid(output).cpu().detach().numpy().tolist()
  print(final_output)

[[0.023955848067998886, 0.9817574620246887, 0.462361603975296, 0.016841810196638107]]
